In [15]:
import os
import pandas as pd
import numpy as np

In [16]:
pwd = os.getcwd()

filepath = pwd + '/Uncleaned_DS_jobs.csv'

raw_data = pd.read_csv(filepath) 

wip = raw_data.copy()

In [17]:
wip.shape

(672, 15)

In [18]:
wip.head()

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Sr Data Scientist,$137K-$171K (Glassdoor est.),Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna"
1,1,Data Scientist,$137K-$171K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
2,2,Data Scientist,$137K-$171K (Glassdoor est.),Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1
3,3,Data Scientist,$137K-$171K (Glassdoor est.),JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON\n3.5,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech..."
4,4,Data Scientist,$137K-$171K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


clean data

In [19]:
# replace \n with delimiter
wip['Company Name'] = wip['Company Name'].str.replace("\n", ",", regex=False)

# remove everything after delimiter
result = []
for item in wip['Company Name']:
    parts = item.split(',')
    result.append(','.join(parts[:1]))

wip['Company Name'] = result

In [20]:
# remove $
wip['SalaryEstUSD'] = wip['Salary Estimate'].str.replace('$', '', regex=False)

# add delimiter
# two formats present '''$145K-$225K (Glassdoor est.)''' and '''$145K-$225K(Employer est.)'''
wip['SalaryEstUSD'] = wip['SalaryEstUSD'].str.replace('(', ':', regex=False)

# remove everything after delimiter
result = []
for item in wip['SalaryEstUSD']:
    parts = item.split(':')
    result.append(':'.join(parts[:1]))

wip['SalaryEstUSD'] = result

In [21]:
# split into new columns
wip[['MinSalaryEstUSD', 'MaxSalaryEstUSD']] = wip['SalaryEstUSD'].str.split("-", expand=True)

# convert values
def convert_values(val):
    if 'K' in val:
        return float(val.replace('K', '')) * 1000
    else:
        return float(val)

wip['MinSalaryEstUSD'] = wip['MinSalaryEstUSD'].apply(convert_values)
wip['MaxSalaryEstUSD'] = wip['MaxSalaryEstUSD'].apply(convert_values)

# convert from float to int
wip['MinSalaryEstUSD'] = wip['MinSalaryEstUSD'].astype(int)
wip['MaxSalaryEstUSD'] = wip['MaxSalaryEstUSD'].astype(int)

In [22]:
# new column, only state job is located in
wip['Job State'] = wip['Location'].str.split(',').str[-1]

# new column, only state headquarters is located in
wip['HQ State'] = wip['Headquarters'].str.split(',').str[-1]

# new column, if job and HQ are located in the same state
wip['Same State'] = wip['Job State'] == wip['HQ State']

In [23]:
# calculate company age
wip['Company Age'] = 2024 - wip['Founded']

# return sentinel values
wip['Company Age'] = wip['Company Age'].where(wip['Company Age'] != 2025, other='-1')

In [24]:
wip.shape

(672, 22)

In [25]:
# remove listings without a rating (-1.0)
wip = wip[wip['Rating'] != -1]

In [26]:
wip.shape

(622, 22)

In [35]:
# find duplicates
wip[wip.duplicated() == True]

,JobTitle,JobDescription,Rating,MinSalaryEstUSD,MaxSalaryEstUSD,Company,Location,Job State,Headquarters,SameState,Size,Founded,CompanyAge,OwnershipType,Industry,Sector,Revenue,Competitors
135,Machine Learning Engineer,Role Description\nTriplebyte screens and evalu...,3.2,90000,109000,Triplebyte,Remote,Remote,"San Francisco, CA",False,51 to 200 employees,2015,9,Company - Private,Computer Hardware & Software,Information Technology,Unknown / Non-Applicable,-1
136,Senior Data Engineer,Lendio is looking to fill a position for a Sen...,4.9,90000,109000,Lendio,"Lehi, UT",UT,"Lehi, UT",True,201 to 500 employees,2011,13,Company - Private,Lending,Finance,$50 to $100 million (USD),-1


In [31]:
# rename columns
wip = wip.rename(columns={'Job Title':'JobTitle', 
                    'Job Description':'JobDescription',
                    'Company Name':'Company',
                    'Same State':'SameState', 
                    'Company Age':'CompanyAge', 
                    'Type of ownership':'OwnershipType'
})

In [32]:
wip.columns

Index(['JobTitle', 'JobDescription', 'Rating', 'MinSalaryEstUSD',
       'MaxSalaryEstUSD', 'Company', 'Location', 'Job State', 'Headquarters',
       'SameState', 'Size', 'Founded', 'CompanyAge', 'OwnershipType',
       'Industry', 'Sector', 'Revenue', 'Competitors'],
      dtype='object')

In [33]:
# reorder columns
clean = wip[['JobTitle', 'JobDescription', 'Rating', 'MinSalaryEstUSD', 'MaxSalaryEstUSD', 'Company', 'Location', 'Job State', 'Headquarters', 'SameState', 'Size', 'Founded', 'CompanyAge', 'OwnershipType', 'Industry', 'Sector', 'Revenue', 'Competitors']]

In [34]:
clean.head()

,JobTitle,JobDescription,Rating,MinSalaryEstUSD,MaxSalaryEstUSD,Company,Location,Job State,Headquarters,SameState,Size,Founded,CompanyAge,OwnershipType,Industry,Sector,Revenue,Competitors
0,Sr Data Scientist,Description\n\nThe Senior Data Scientist is re...,3.1,137000,171000,Healthfirst,"New York, NY",NY,"New York, NY",True,1001 to 5000 employees,1993,31,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna"
1,Data Scientist,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,137000,171000,ManTech,"Chantilly, VA",VA,"Herndon, VA",True,5001 to 10000 employees,1968,56,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
2,Data Scientist,Overview\n\n\nAnalysis Group is one of the lar...,3.8,137000,171000,Analysis Group,"Boston, MA",MA,"Boston, MA",True,1001 to 5000 employees,1981,43,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1
3,Data Scientist,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,137000,171000,INFICON,"Newton, MA",MA,"Bad Ragaz, Switzerland",False,501 to 1000 employees,2000,24,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech..."
4,Data Scientist,Data Scientist\nAffinity Solutions / Marketing...,2.9,137000,171000,Affinity Solutions,"New York, NY",NY,"New York, NY",True,51 to 200 employees,1998,26,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"
